# Runnables

To make it as easy as possible to create custom chains, LangChain has implemented a "Runnable" protocol. Many LangChain components implement the Runnable protocol, including chat models, LLMs, output parsers, retrievers, prompt templates, and more.

The main composition primitives are RunnableSequence and RunnableParallel.

- *RunnableSequence* invokes a series of runnables sequentially, with one Runnable’s output serving as the next’s input. Construct using the | operator or by passing a list of runnables to *RunnableSequence*.

- *RunnableParallel* invokes runnables concurrently, providing the same input to each. Construct it using a dict literal within a sequence or by passing a dict to RunnableParallel.

Ref: https://python.langchain.com/v0.1/docs/expression_language/interface/

In [2]:
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnableSequence

## RunnableLambda()

In [30]:
# Runnable Lambda with stream

sequence = RunnableLambda(lambda x: x*5)
output = sequence.stream(4)

for chunk in output:
    print(chunk)

20


In [15]:
# Runnable Lambda with stream

sequence = RunnableLambda(lambda x: x*5)
output = sequence.invoke(4)
print(output)

20


In [31]:
# Runnable Lambda with batch

sequence = RunnableLambda(lambda x: x*5)
output = sequence.batch([1,2,3,4])
print(output)

[5, 10, 15, 20]


Please note: RunnableLambdas do not support transform by default!

In [48]:
# Runnable Lambda with transform

sequence = RunnableLambda(lambda x: x*5)
output = sequence.transform(4)
print(output)

<generator object RunnableLambda.transform at 0x000001A772444040>


## RunnableSequence()

Ref: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.base.RunnableSequence.html

In [44]:
from langchain_core.runnables import RunnableLambda

def add_one(x: int) -> int:
    return x + 1

def mul_two(x: int) -> int:
    return x * 2

runnable_1 = RunnableLambda(add_one)
runnable_2 = RunnableLambda(mul_two)
sequence = runnable_1 | runnable_2

In [45]:
%%time
sequence.invoke(1)

CPU times: total: 0 ns
Wall time: 3.83 ms


4

In [47]:
sequence = RunnableSequence(first=runnable_1, last=runnable_2)
sequence.invoke(1)

4

In [46]:
async def add_one(x: int) -> int:
    return x + 1

async def mul_two(x: int) -> int:
    return x * 2

await sequence.ainvoke(1) #cant use await without async

4

## RunnableParallel()

Ref: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.base.RunnableParallel.html

### Creating a combination of sequential and parallel runnables for simple Math transformation

In [32]:
# Method 1

add_2 = RunnableLambda(lambda x: x+2)
mul_2 = RunnableLambda(lambda x: x*2)

chain = RunnableLambda(lambda y: y-2) | RunnableParallel(fun1 = add_2, fun2 = mul_2)
chain.invoke(3)

{'fun1': 3, 'fun2': 2}

In [33]:
# Method 2

chain = RunnableLambda(lambda y: y-2) | RunnableParallel(
    {
        "fun1" : add_2, 
        "fun2" : mul_2
    }
    )
chain.invoke(3)

{'fun1': 3, 'fun2': 2}

In [34]:
# Method 3

chain = RunnableLambda(lambda y: y-2) | {
        "fun1" : add_2, 
        "fun2" : mul_2
    }
chain.invoke(3)

{'fun1': 3, 'fun2': 2}

### Calling a Runnable within a Runnable

In [35]:
def my_lambda(x):
    return RunnableLambda(lambda y: y * 2)

outer_lambda = RunnableLambda(my_lambda)

output = outer_lambda.invoke(5)

print(output)

10
